
Data were prepared by [this script](https://www.kaggle.com/aharless/preparing-data-for-lgbm-or-something-else/output).

This is essentially a 2-step process to implement (Bojan's [retuning](https://www.kaggle.com/tunguz/lgbm-one-step-ahead-lb-0-514) of) [Lingzhi's upgraded version](https://www.kaggle.com/vrtjso/lgbm-one-step-ahead) of Ceshine's [LGBM starter](https://www.kaggle.com/ceshine/lgbm-starter) script.  The results are slightly different, probably because of lost precision (since I only use 6 decimal places in the prepared data) or maybe because there's a bug.

This is an experiment with the new Kaggle feature that allows you to create kernels directly using the output of another kernel.   If you want to run variations on the model fit (or a completely different model using the same data setup), you don't have to prepare the data again every time.  Turns out it's pretty easy to [run a different kind of model](https://www.kaggle.com/aharless/nn-starter-using-lingzhi-data/) on the same data.

In [1]:
MAX_PRED=1000

In [2]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

favorita-grocery-sales-forecasting
preparing-data-for-lgbm-or-something-else



In [3]:
indir = '../input/preparing-data-for-lgbm-or-something-else/'
indir2 = '../input/favorita-grocery-sales-forecasting/'

In [4]:
X_test = pd.read_csv(indir + 'X_test.csv')
X_val = pd.read_csv(indir + 'X_val.csv')
X_train = pd.read_csv(indir + 'X_train.csv')
y_train = np.array(pd.read_csv(indir + 'y_train.csv'))
y_val = np.array(pd.read_csv(indir + 'y_val.csv'))
stores_items = pd.read_csv(indir + 'stores_items.csv', index_col=['store_nbr','item_nbr'])
test_ids = pd.read_csv( indir + 'test_ids.csv',  parse_dates=['date']).set_index(
                        ['store_nbr', 'item_nbr', 'date'] )

In [5]:
items = pd.read_csv( indir2 + 'items.csv' ).set_index("item_nbr")
items = items.reindex( stores_items.index.get_level_values(1) )

In [6]:
params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

In [7]:
MAX_ROUNDS = 3000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=500
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

Step 1


/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1030: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds.
[500]	training's l2: 0.301319	valid_1's l2: 0.29358
[1000]	training's l2: 0.297485	valid_1's l2: 0.292137
[1500]	training's l2: 0.294849	valid_1's l2: 0.291689
[2000]	training's l2: 0.292608	valid_1's l2: 0.291455
[2500]	training's l2: 0.29059	valid_1's l2: 0.291284
[3000]	training's l2: 0.288713	valid_1's l2: 0.291182
mean_7_2017: 8837494.87
mean_14_2017: 5724666.77
mean_30_2017: 730472.62
promo_0: 510501.74
day_1_2017: 430153.05
mean_3_2017: 419690.21
mean_20_dow0_2017: 419304.59
mean_4_dow0_2017: 275115.89
promo_14_2017: 136050.21
mean_60_2017: 118814.88
mean_4_dow5_2017: 41869.78
promo_7: 39819.89
mean_140_2017: 37296.20
promo_60_2017: 34707.96
promo_140_2017: 31761.05
mean_4_dow6_2017: 29343.16
mean_20_dow4_2017: 28911.88
mean_4_dow2_2017: 25342.88
mean_20_dow2_2017: 23826.37
mean_20_dow1_2017: 16954.39
mean_4_dow1_2017: 16533.90
promo_14: 16274.59
mean_4_dow4_2017: 16187.63
mean_20_dow3_2017: 15631.91
mean_4_dow3_2017

In [8]:
n_public = 5 # Number of days in public test set
weights=pd.concat([items["perishable"]]) * 0.25 + 1
print("Unweighted validation mse: ", mean_squared_error(
    y_val, np.minimum( np.array(val_pred).transpose(), np.log1p(MAX_PRED) ) )   )
mse = mean_squared_error(
    y_val, np.minimum( np.array(val_pred).transpose(), np.log1p(MAX_PRED) ), 
    sample_weight=weights)
print("Full validation mse:       ", mse )
msepub = mean_squared_error(
    y_val[:,:n_public], 
    np.minimum( np.array(val_pred).transpose()[:,:n_public], np.log1p(MAX_PRED) ),
    sample_weight=weights)
print("'Public' validation mse:   ",  msepub )
msepriv = mean_squared_error(
    y_val[:,n_public:], 
    np.minimum( np.array(val_pred).transpose()[:,n_public:], np.log1p(MAX_PRED) ),
    sample_weight=weights)
print("'Private' validation mse:  ",  msepriv )
print('Validation NRMSWLE')
print( "  Full:    ", np.sqrt(mse) )
print( "  Public:  ", np.sqrt(msepub) )
print( "  Private: ", np.sqrt(msepriv) )

Unweighted validation mse:  0.361791131762
Full validation mse:        0.361260891897
'Public' validation mse:    0.331851350717
'Private' validation mse:   0.37462886516
Validation NRMSWLE
  Full:     0.601049824804
  Public:   0.576065404895
  Private:  0.612069330354


In [9]:
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=stores_items.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [10]:
submission = test_ids.join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, MAX_PRED)
submission.to_csv('lgb_whatever.csv', float_format='%.4f', index=None)